In [1]:
!pip install mediapipe opencv-python pandas scikit-learn

In [2]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv

In [3]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

In [6]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        # FACEMESH_TESSELATION
        """
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
                                 """
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [5]:
#results.face_landmarks

In [6]:
#results.face_landmarks.landmark[0].visibility

In [7]:
import csv
import os
import numpy as np

In [8]:
num_coords = len(results.pose_landmarks.landmark)
#+len(results.face_landmarks.landmark)
num_coords

33

In [9]:
landmarks = ['class']
#추출할 csv 형식 
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [10]:
#landmarks
landmarks[-1]

'v33'

In [72]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [38]:
class_name = "Standing"

In [39]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            
            
            # Concate rows
            row = pose_row
            #+face_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [13]:
df = pd.read_csv('coords.csv')

In [14]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,Standing,0.551715,0.411068,-0.480280,0.999614,0.560207,0.390554,-0.463207,0.999229,0.565768,...,0.424125,0.045544,0.590840,1.245772,0.220937,0.035616,0.512582,1.247243,0.246170,0.037159
1,Standing,0.550886,0.411065,-0.501023,0.999620,0.560119,0.390556,-0.481965,0.999244,0.565779,...,0.516370,0.043974,0.588501,1.269854,0.251413,0.033711,0.513790,1.278817,0.316399,0.035369
2,Standing,0.550391,0.410203,-0.497141,0.999628,0.560053,0.389839,-0.478470,0.999259,0.565798,...,0.542966,0.042160,0.587419,1.277163,0.264595,0.031328,0.514399,1.285124,0.345723,0.033125
3,Standing,0.550295,0.409257,-0.498234,0.999636,0.560080,0.389303,-0.478785,0.999273,0.565927,...,0.542878,0.040295,0.585506,1.282257,0.274774,0.029196,0.514306,1.289704,0.342759,0.031088
4,Standing,0.550293,0.408860,-0.496996,0.999631,0.560187,0.389166,-0.477702,0.999258,0.566151,...,0.528384,0.038137,0.585423,1.282915,0.262519,0.027317,0.514810,1.290418,0.330170,0.029151


In [15]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
17484,Standing,0.486214,0.300045,-0.874068,0.999838,0.509706,0.255652,-0.840441,0.999633,0.521378,...,0.404363,0.000080,0.585448,2.243240,0.179610,0.000144,0.432656,2.244449,-0.033118,0.000078
17485,Standing,0.486602,0.298456,-0.839846,0.999842,0.509934,0.254176,-0.806274,0.999641,0.521627,...,0.405948,0.000078,0.585611,2.243124,0.175706,0.000141,0.432918,2.244493,-0.035026,0.000077
17486,Standing,0.486593,0.297454,-0.854455,0.999842,0.509953,0.253543,-0.820983,0.999639,0.521663,...,0.422418,0.000077,0.585391,2.241732,0.184138,0.000139,0.432578,2.243686,-0.021181,0.000076
17487,Standing,0.486609,0.297580,-0.901225,0.999836,0.509952,0.253609,-0.866816,0.999626,0.521658,...,0.524639,0.000079,0.587932,2.238812,0.247221,0.000141,0.434537,2.242043,0.071132,0.000078
17488,Standing,0.486063,0.297671,-0.962774,0.999830,0.509682,0.253670,-0.929679,0.999612,0.521312,...,0.505106,0.000078,0.587878,2.240331,0.235540,0.000140,0.434538,2.244111,0.050971,0.000078


In [16]:
df[df['class']=='Standing']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,Standing,0.551715,0.411068,-0.480280,0.999614,0.560207,0.390554,-0.463207,0.999229,0.565768,...,0.424125,0.045544,0.590840,1.245772,0.220937,0.035616,0.512582,1.247243,0.246170,0.037159
1,Standing,0.550886,0.411065,-0.501023,0.999620,0.560119,0.390556,-0.481965,0.999244,0.565779,...,0.516370,0.043974,0.588501,1.269854,0.251413,0.033711,0.513790,1.278817,0.316399,0.035369
2,Standing,0.550391,0.410203,-0.497141,0.999628,0.560053,0.389839,-0.478470,0.999259,0.565798,...,0.542966,0.042160,0.587419,1.277163,0.264595,0.031328,0.514399,1.285124,0.345723,0.033125
3,Standing,0.550295,0.409257,-0.498234,0.999636,0.560080,0.389303,-0.478785,0.999273,0.565927,...,0.542878,0.040295,0.585506,1.282257,0.274774,0.029196,0.514306,1.289704,0.342759,0.031088
4,Standing,0.550293,0.408860,-0.496996,0.999631,0.560187,0.389166,-0.477702,0.999258,0.566151,...,0.528384,0.038137,0.585423,1.282915,0.262519,0.027317,0.514810,1.290418,0.330170,0.029151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17484,Standing,0.486214,0.300045,-0.874068,0.999838,0.509706,0.255652,-0.840441,0.999633,0.521378,...,0.404363,0.000080,0.585448,2.243240,0.179610,0.000144,0.432656,2.244449,-0.033118,0.000078
17485,Standing,0.486602,0.298456,-0.839846,0.999842,0.509934,0.254176,-0.806274,0.999641,0.521627,...,0.405948,0.000078,0.585611,2.243124,0.175706,0.000141,0.432918,2.244493,-0.035026,0.000077
17486,Standing,0.486593,0.297454,-0.854455,0.999842,0.509953,0.253543,-0.820983,0.999639,0.521663,...,0.422418,0.000077,0.585391,2.241732,0.184138,0.000139,0.432578,2.243686,-0.021181,0.000076
17487,Standing,0.486609,0.297580,-0.901225,0.999836,0.509952,0.253609,-0.866816,0.999626,0.521658,...,0.524639,0.000079,0.587932,2.238812,0.247221,0.000141,0.434537,2.242043,0.071132,0.000078


In [17]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [18]:
df['class']

0        Standing
1        Standing
2        Standing
3        Standing
4        Standing
           ...   
17484    Standing
17485    Standing
17486    Standing
17487    Standing
17488    Standing
Name: class, Length: 17489, dtype: object

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [20]:
X_test

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
12550,0.613504,0.178352,-0.429260,0.999906,0.625413,0.153254,-0.395968,0.999841,0.632625,0.155451,...,0.402516,0.012417,0.686813,1.469835,0.222266,0.011703,0.579668,1.478819,0.136425,0.007966
3781,0.723037,0.102106,-0.353585,0.999962,0.731183,0.084194,-0.334519,0.999920,0.737278,0.083541,...,-0.097734,0.816313,0.726437,0.980809,-0.238165,0.875753,0.474390,0.818176,-0.274812,0.951664
7294,0.558712,0.288907,-0.654115,0.999224,0.571469,0.262186,-0.631031,0.998553,0.579641,0.262612,...,0.632119,0.039577,0.634159,1.493152,0.249286,0.017789,0.542202,1.492084,0.367858,0.019083
3986,0.715880,0.108365,-0.391100,0.999942,0.724478,0.089991,-0.371938,0.999878,0.730512,0.089566,...,-0.062257,0.785894,0.723563,0.974748,-0.232816,0.842678,0.476530,0.835343,-0.238589,0.929990
13476,0.518655,0.356684,-0.517450,0.999941,0.539774,0.318102,-0.461183,0.999838,0.550500,0.320672,...,0.206194,0.000120,0.602875,2.305221,-0.125424,0.000191,0.463497,2.292279,-0.239703,0.000140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6748,0.575714,0.281661,-0.707973,0.999034,0.587695,0.253508,-0.687309,0.998049,0.596275,0.253236,...,0.644030,0.056118,0.643589,1.466609,0.239983,0.035036,0.534687,1.468918,0.369871,0.033120
14029,0.625713,0.193542,-0.356680,0.999993,0.633707,0.177577,-0.341812,0.999977,0.637650,0.177968,...,0.168444,0.806333,0.672773,0.945365,-0.071433,0.981182,0.623219,0.946306,0.007713,0.970296
7097,0.563622,0.283341,-0.661315,0.999320,0.578131,0.255581,-0.637939,0.998722,0.586958,0.255889,...,0.598857,0.056194,0.633610,1.483835,0.243751,0.029728,0.537827,1.488692,0.325293,0.027755
5036,0.568593,0.081327,-0.228071,0.999938,0.572184,0.061966,-0.203556,0.999839,0.575086,0.063021,...,-0.016055,0.753296,0.897307,0.791140,-0.162626,0.928164,0.590915,0.960578,-0.172589,0.935893


In [21]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [22]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [23]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

C:\Users\seoeu\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [24]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [25]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

In [26]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 0.9971412235563178
rc 0.9918048408614446
rf 1.0
gb 0.9986659043262817


In [27]:
fit_models['lr'].predict(X_test)

array(['Fighting_L', 'Kicking_R', 'Standing', ..., 'Standing',
       'Kicking_L', 'Standing'], dtype=object)

In [28]:
y_test

12550    Fighting_L
3781      Kicking_R
7294       Standing
3986      Kicking_R
13476    Fighting_L
            ...    
6748       Standing
14029      Standing
7097       Standing
5036      Kicking_L
14148      Standing
Name: class, Length: 5247, dtype: object

In [29]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['gb'], f)

In [30]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [31]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('gradientboostingclassifier', GradientBoostingClassifier())])

In [32]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        """
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
                                  """
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
    
                
            # Concate rows
            row = pose_row
            #+face_row
            
#             # Append class name 
#             row.insert(0, class_name)
            
#             # Export to CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row) 

            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

Fighting_R [2.74088118e-05 8.75668880e-01 3.16190293e-07 7.81793295e-07
 1.24302614e-01]
Fighting_R [1.99750996e-05 8.96242263e-01 3.23917587e-07 8.00899342e-07
 1.03736637e-01]
Fighting_R [1.95054643e-05 9.04424840e-01 3.16301948e-07 7.82069366e-07
 9.55545558e-02]
Fighting_R [1.95054643e-05 9.04424840e-01 3.16301948e-07 7.82069366e-07
 9.55545558e-02]
Fighting_R [1.95054643e-05 9.04424840e-01 3.16301948e-07 7.82069366e-07
 9.55545558e-02]
Fighting_R [1.95054643e-05 9.04424840e-01 3.16301948e-07 7.82069366e-07
 9.55545558e-02]
Fighting_R [1.93940816e-05 8.99260276e-01 3.14495759e-07 7.77603492e-07
 1.00719238e-01]
Fighting_R [1.93940816e-05 8.99260276e-01 3.14495759e-07 7.77603492e-07
 1.00719238e-01]
Fighting_R [1.93940816e-05 8.99260276e-01 3.14495759e-07 7.77603492e-07
 1.00719238e-01]
Fighting_R [1.93940816e-05 8.99260276e-01 3.14495759e-07 7.77603492e-07
 1.00719238e-01]
Fighting_R [1.84697072e-05 9.04061804e-01 2.99506039e-07 7.40540803e-07
 9.59186859e-02]
Fighting_R [1.8520345

Fighting_R [2.27300397e-05 8.93383363e-01 3.57596442e-07 8.84171675e-07
 1.06592665e-01]
Fighting_R [2.25880013e-05 8.87800674e-01 3.55361848e-07 8.78646549e-07
 1.12175504e-01]
Fighting_R [2.21182417e-05 8.75307511e-01 3.47971435e-07 8.60373455e-07
 1.24669163e-01]
Fighting_R [2.20916440e-05 8.75457456e-01 3.47445539e-07 8.59338835e-07
 1.24519245e-01]
Fighting_R [2.20652015e-05 8.75606871e-01 3.47029665e-07 5.14672271e-07
 1.24370202e-01]
Fighting_R [2.20739374e-05 8.75557622e-01 3.47167059e-07 5.14876037e-07
 1.24419442e-01]
Fighting_R [2.20739374e-05 8.75557622e-01 3.47167059e-07 5.14876037e-07
 1.24419442e-01]
Fighting_R [3.10286706e-05 8.75549781e-01 3.47163950e-07 5.14871426e-07
 1.24418328e-01]
Fighting_R [3.08338267e-05 8.76331264e-01 3.44983942e-07 5.11638303e-07
 1.23637046e-01]
Fighting_R [3.08587529e-05 8.76231290e-01 3.45262828e-07 5.12051912e-07
 1.23736994e-01]
Fighting_R [3.35166224e-05 8.76328914e-01 3.43486812e-07 5.11636931e-07
 1.23636714e-01]
Fighting_R [3.3178685

Fighting_R [5.24399885e-04 9.70158652e-01 8.23569537e-06 1.22316617e-05
 2.92964809e-02]
Fighting_R [5.17710331e-04 9.75090428e-01 8.13063599e-06 1.20756274e-05
 2.43716552e-02]
Fighting_R [5.11279579e-04 9.75399843e-01 8.02964109e-06 1.19256297e-05
 2.40689221e-02]
Fighting_R [5.00547187e-04 9.56149487e-01 7.86108897e-06 1.16752959e-05
 4.33304294e-02]
Fighting_R [5.03407719e-04 9.61613699e-01 7.90601359e-06 1.17420180e-05
 3.78632450e-02]
Fighting_R [5.03412781e-04 9.61613313e-01 7.90609309e-06 1.17421361e-05
 3.78636257e-02]
Fighting_R [5.06540398e-04 9.66348507e-01 7.95521229e-06 1.18150880e-05
 3.31251826e-02]
Fighting_R [4.50595084e-04 9.67847929e-01 7.07659165e-06 1.05101599e-05
 3.16838896e-02]
Fighting_R [4.50595084e-04 9.67847929e-01 7.07659165e-06 1.05101599e-05
 3.16838896e-02]
Fighting_R [4.52222922e-04 9.70449949e-01 7.10215683e-06 1.05481293e-05
 2.90801781e-02]
Fighting_R [4.50045674e-04 9.71155503e-01 7.06796316e-06 1.04973448e-05
 2.83768858e-02]
Fighting_R [4.4912407

Fighting_L [8.72435652e-01 2.71365341e-04 2.18856725e-06 6.34203298e-06
 1.27284452e-01]
Standing [1.09261560e-02 2.07625107e-03 1.68993277e-05 4.86558905e-05
 9.86932038e-01]
Standing [8.36727307e-03 1.37803711e-04 1.11980778e-06 2.79385968e-06
 9.91491010e-01]
Standing [8.10808486e-03 1.37839729e-04 1.12301020e-06 2.79458991e-06
 9.91750158e-01]
Standing [2.24828318e-02 1.55504156e-04 2.13195620e-06 5.27026365e-06
 9.77354262e-01]
Standing [3.40405941e-04 7.08999800e-05 9.71552583e-07 1.64011515e-05
 9.99571321e-01]
Standing [2.98793330e-04 7.03476797e-05 9.63984333e-07 3.03347735e-05
 9.99599560e-01]
Standing [2.92297839e-04 7.01932585e-05 9.61868278e-07 2.04684895e-06
 9.99634500e-01]
Standing [5.52440102e-04 8.57874948e-05 1.17614337e-06 1.74341507e-06
 9.99358853e-01]
Standing [3.33139342e-04 8.42614683e-05 1.15522160e-06 1.71375786e-06
 9.99579730e-01]
Standing [2.39415424e-04 4.94332475e-05 6.77727987e-07 1.00540162e-06
 9.99709468e-01]
Standing [9.92870884e-04 1.24277075e-04 1

Fighting_R [2.22830998e-04 9.90657028e-01 3.49955877e-06 5.19754763e-06
 9.11144428e-03]
Fighting_R [2.23120179e-04 9.91942664e-01 3.50410036e-06 5.20429280e-06
 7.82550779e-03]
Standing [1.09450986e-04 2.26254871e-04 1.72191882e-06 2.55295143e-06
 9.99660019e-01]
Standing [9.50773662e-05 1.09038427e-04 1.49578832e-06 2.21768580e-06
 9.99792171e-01]
Standing [9.50773662e-05 1.09038427e-04 1.49578832e-06 2.21768580e-06
 9.99792171e-01]
Standing [8.78423081e-05 1.00740981e-04 1.38196401e-06 2.04892759e-06
 9.99807986e-01]
Standing [8.78423081e-05 1.00740981e-04 1.38196401e-06 2.04892759e-06
 9.99807986e-01]
Standing [8.78423081e-05 1.00740981e-04 1.38196401e-06 2.04892759e-06
 9.99807986e-01]
Standing [8.78423081e-05 1.00740981e-04 1.38196401e-06 2.04892759e-06
 9.99807986e-01]
Standing [8.78423081e-05 1.00740981e-04 1.38196401e-06 2.04892759e-06
 9.99807986e-01]
Standing [8.78423081e-05 1.00740981e-04 1.38196494e-06 2.04892759e-06
 9.99807986e-01]
Standing [8.78423081e-05 1.00740981e-04

...

Standing [1.91417764e-04 1.03875837e-04 1.42310416e-06 2.11268610e-06
 9.99701171e-01]
Standing [1.91417764e-04 1.03875837e-04 1.42310416e-06 2.11268610e-06
 9.99701171e-01]
Standing [1.91417765e-04 1.03875838e-04 1.41722086e-06 2.11268611e-06
 9.99701176e-01]
Standing [1.91417765e-04 1.03875838e-04 1.41722086e-06 2.11268611e-06
 9.99701176e-01]
Standing [1.91417765e-04 1.03875838e-04 1.41722086e-06 2.11268611e-06
 9.99701176e-01]
Standing [1.91417765e-04 1.03875838e-04 1.41722086e-06 2.11268611e-06
 9.99701176e-01]
Standing [1.91417765e-04 1.03875838e-04 1.41722086e-06 2.11268611e-06
 9.99701176e-01]
Standing [1.19236383e-04 1.36744929e-04 1.86521215e-06 2.78119645e-06
 9.99739372e-01]
Standing [1.19236383e-04 1.36744928e-04 1.86566741e-06 2.78119645e-06
 9.99739372e-01]
Standing [9.05849193e-05 1.03886314e-04 1.41736379e-06 2.11289918e-06
 9.99801999e-01]
Standing [9.76354836e-05 1.11972176e-04 1.53449882e-06 2.27735406e-06
 9.99786580e-01]
Standing [9.76354835e-05 1.11972176e-04 1.5

Fighting_R [4.56100191e-02 8.81969480e-01 1.25668907e-04 1.83028219e-04
 7.21118042e-02]
Fighting_R [1.50421870e-01 8.02914344e-01 1.14404603e-04 1.66622526e-04
 4.63827590e-02]
Fighting_R [1.50421870e-01 8.02914344e-01 1.14404603e-04 1.66622526e-04
 4.63827590e-02]
Fighting_R [1.48378031e-01 7.92004843e-01 1.12850145e-04 1.64358563e-04
 5.93399181e-02]
Fighting_R [1.46395255e-01 7.81421283e-01 1.11342128e-04 1.62162239e-04
 7.19099572e-02]
Fighting_R [9.46673806e-03 9.60942657e-01 6.32543075e-06 1.18476322e-05
 2.95724318e-02]
Standing [5.79156238e-02 3.88151627e-03 3.11283677e-05 5.82371564e-05
 9.38113494e-01]
Standing [3.90335991e-01 1.06668520e-02 8.55443254e-05 2.07053581e-04
 5.98704559e-01]
Standing [2.80160502e-01 3.74948807e-03 5.04875273e-05 1.09729737e-03
 7.14942225e-01]
Standing [5.60466559e-02 2.18751400e-03 2.94552672e-05 1.13297793e-02
 9.30406596e-01]
Kicking_R [1.33825892e-02 1.79816330e-03 2.44329218e-05 8.84558940e-01
 1.00235875e-01]
Kicking_R [5.42852508e-06 7.29

Standing [1.85499354e-01 1.16108133e-01 2.31789849e-04 1.32795702e-03
 6.96832766e-01]
Standing [1.82590273e-01 1.16484823e-01 2.32541844e-04 1.33226531e-03
 6.99360098e-01]
Standing [2.99887688e-01 1.90231966e-01 3.79765289e-04 7.11744600e-04
 5.08788836e-01]
Standing [0.30418149 0.3009506  0.00059414 0.00086654 0.39340724]
Fighting_L [0.44259608 0.23677972 0.0004638  0.00067645 0.31948395]
Standing [0.20529183 0.38577849 0.00075566 0.00110212 0.4070719 ]
Standing [0.12761776 0.42348414 0.00082952 0.00120984 0.44685874]
Standing [0.07674161 0.43820488 0.0008522  0.00124291 0.48295841]
Standing [0.07674161 0.43820488 0.0008522  0.00124291 0.48295841]
Standing [0.07594108 0.43363372 0.00084197 0.00122645 0.48835678]
Standing [0.07428227 0.42416175 0.00082358 0.00119966 0.49953274]
Standing [0.07375456 0.42114842 0.00081773 0.00119114 0.50308816]
Standing [0.07476428 0.44241041 0.00082892 0.00120727 0.48078912]
Standing [0.07476428 0.44241041 0.00082892 0.00120727 0.48078912]
Fighting_R 

Standing [9.99636610e-04 8.25835434e-04 1.11200218e-05 1.61901782e-05
 9.98147218e-01]
Standing [9.99636610e-04 8.25835434e-04 1.11200218e-05 1.61901782e-05
 9.98147218e-01]
Standing [9.99636610e-04 8.25835434e-04 1.11200218e-05 1.61901782e-05
 9.98147218e-01]
Standing [9.99636610e-04 8.25835434e-04 1.11200218e-05 1.61901782e-05
 9.98147218e-01]
Standing [9.99636610e-04 8.25835434e-04 1.11200218e-05 1.61901782e-05
 9.98147218e-01]
Standing [9.99636610e-04 8.25835434e-04 1.11200218e-05 1.61901782e-05
 9.98147218e-01]
Standing [9.99636610e-04 8.25835434e-04 1.11200218e-05 1.61901782e-05
 9.98147218e-01]
Standing [9.99636610e-04 8.25835434e-04 1.11200218e-05 1.61901782e-05
 9.98147218e-01]
Standing [9.99636610e-04 8.25835434e-04 1.11200218e-05 1.61901782e-05
 9.98147218e-01]
Standing [9.99636610e-04 8.25835434e-04 1.11200218e-05 1.61901782e-05
 9.98147218e-01]
Standing [1.00879770e-03 8.33403737e-04 1.12219303e-05 1.63385517e-05
 9.98130238e-01]
Standing [1.00879770e-03 8.33403737e-04 1.1

Fighting_L [9.44928033e-01 5.12929913e-02 5.39037381e-06 9.30333074e-06
 3.76428227e-03]
Fighting_L [9.44315356e-01 5.08235690e-02 6.76942021e-06 1.00952709e-05
 4.84421077e-03]
Fighting_L [9.54109838e-01 4.12083139e-02 6.51983190e-06 9.72305862e-06
 4.66560487e-03]
Fighting_L [9.53635309e-01 4.11878188e-02 6.51658924e-06 1.12470836e-05
 5.15910894e-03]
Fighting_L [9.81561011e-01 9.73709678e-04 7.86268023e-06 1.35726229e-05
 1.74438440e-02]
Fighting_L [9.81561011e-01 9.73709678e-04 7.86268023e-06 1.35726229e-05
 1.74438440e-02]
Fighting_L [9.68351659e-01 1.22505561e-03 9.89229204e-06 1.70761554e-05
 3.03963170e-02]
Fighting_L [9.59142506e-01 2.90383454e-03 9.85117105e-06 1.69902218e-05
 3.79268183e-02]
Fighting_L [9.59089488e-01 2.90760262e-03 9.86395415e-06 1.70122687e-05
 3.79760330e-02]
Fighting_L [7.88155002e-01 1.98377136e-01 8.10594308e-06 1.39802436e-05
 1.34457762e-02]
Fighting_L [6.73665317e-01 3.14602410e-01 7.07478799e-06 1.22055937e-05
 1.17129926e-02]
Fighting_L [6.3606287